In [1]:
import pyspark
from pyspark.sql import SparkSession

master_url = 'spark://de-zoomcamp.asia-northeast3-c.c.de-zoomcamp-joshua.internal:7077'

spark = SparkSession.builder \
    .master(master_url) \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/02/01 13:40:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


* Go back to [http://localhost:8080](http://localhost:8080), then you'll see a new application is running.

In [2]:
spark

* Go back to `instruction-that-i-wrote-up.md` starting from **Run Workers** section.

In [7]:
green_folderpath = 'data/pq/green/*/*'
yellow_folderpath = 'data/pq/yellow/*/*'

df_yellow = spark.read \
    .parquet(yellow_folderpath) \
    .selectExpr(
        "VendorID",
        "tpep_pickup_datetime",
        "tpep_dropoff_datetime",
        "passenger_count",
        "trip_distance",
        "RatecodeID",
        "store_and_fwd_flag",
        "PULocationID",
        "DOLocationID",
        "payment_type",
        "fare_amount",
        "extra",
        "mta_tax",
        "tip_amount",
        "tolls_amount",
        "improvement_surcharge",
        "total_amount",
        "congestion_surcharge",
        "airport_fee"
    )

df_green = spark \
    .read.parquet(green_folderpath) \
    .selectExpr(
        "VendorID",
        "lpep_pickup_datetime",
        "lpep_dropoff_datetime",
        "passenger_count",
        "trip_distance",
        "RatecodeID",
        "store_and_fwd_flag",
        "PULocationID",
        "DOLocationID",
        "payment_type",
        "fare_amount",
        "extra",
        "mta_tax",
        "tip_amount",
        "tolls_amount",
        "improvement_surcharge",
        "total_amount",
        "congestion_surcharge"
    )

In [8]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [9]:
common_cols = []

for item in set(df_yellow.columns) & set(df_green.columns):
    common_cols.append(item)

In [10]:
from pyspark.sql import functions as F

df_yellow_common = df_yellow \
    .select(common_cols) \
    .withColumn('service_type', F.lit('yellow'))

df_green_common = df_green \
    .select(common_cols) \
    .withColumn('service_type', F.lit('green'))

In [11]:
df_all = df_yellow_common.unionAll(df_green_common)

In [12]:
df_all.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|      yellow|75811575|
|       green| 1393284|
+------------+--------+



In [13]:
df_all.columns

['DOLocationID',
 'congestion_surcharge',
 'RatecodeID',
 'total_amount',
 'VendorID',
 'extra',
 'mta_tax',
 'fare_amount',
 'tip_amount',
 'passenger_count',
 'trip_distance',
 'store_and_fwd_flag',
 'payment_type',
 'dropoff_datetime',
 'tolls_amount',
 'improvement_surcharge',
 'PULocationID',
 'pickup_datetime',
 'service_type']

In [14]:
df_all.createOrReplaceTempView('trips_data')

In [15]:
query = """
    SELECT
        service_type,
        COUNT(1) AS cnt
    FROM
        trips_data
    GROUP BY
        1
"""

spark.sql(query).show()

+------------+--------+
|service_type|     cnt|
+------------+--------+
|      yellow|75811575|
|       green| 1393284|
+------------+--------+




[Stage 9:=================================================>       (21 + 3) / 24]



In [16]:
query = """

SELECT 
    PULocationID AS revenue_zone,
    DATE_TRUNC('MONTH', pickup_datetime) AS yyyy_mm,
    service_type,

    SUM(fare_amount) AS fare_amount,
    SUM(extra) AS extra,
    SUM(mta_tax) AS mta_tax,
    SUM(tip_amount) AS tip_amount,
    SUM(tolls_amount) AS tolls_amount,
    SUM(improvement_surcharge) AS improvement_surcharge,
    SUM(total_amount) AS total_amount,
    SUM(congestion_surcharge) AS congestion_surcharge,
    AVG(passenger_count) AS avg_passenger_cnt,
    AVG(trip_distance) AS avg_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
"""

df_result = spark.sql(query)

In [17]:
result_folderpath = 'data/report/revenue/'

# coalesce(1): Decrease the Number of Partitions to 1
df_result.coalesce(1).write.parquet(result_folderpath, mode='overwrite')